In [21]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv("preprocessed-datasets/CongressionVoting_prepro.csv")

df['class'] = df['class'].map({'democrat': 0, 'republican': 1})

# Convert to numpy array
data = df.to_numpy()

# Split the data into training (80%) and testing (20%) sets
np.random.shuffle(data)
split = int(0.8 * len(data))
train_set = data[:split]
test_set = data[split:]

# Split training set to X and y
X_train = train_set[:, :-1]
y_train = train_set[:, -1]

# Split testing set to X and y
X_test = test_set[:, :-1]
y_test = test_set[:, -1]

In [23]:
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

# def sigmoid_derivative(x):
#     return x * (1 - x)

# def initialize_parameters(input_size, hidden_size, output_size):
#     np.random.seed(18)
#     W_h = np.random.rand(input_size, hidden_size)
#     W_o = np.random.rand(hidden_size, output_size)
#     return W_h, W_o

# def forward_propagation(X, W_h, W_o):
#     hidden_layer_input = np.dot(X, W_h)
#     hidden_layer_output = sigmoid(hidden_layer_input)

#     output_layer_input = np.dot(hidden_layer_output, W_o)
#     output_layer_output = sigmoid(output_layer_input)

#     return hidden_layer_output, output_layer_output

# def backward_propagation(X, y, hidden_layer_output, output_layer_output, W_o):
#     output_error = y - output_layer_output
#     output_delta = output_error * sigmoid_derivative(output_layer_output)

#     hidden_layer_error = output_delta.dot(W_o.T)
#     hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer_output)

#     return hidden_layer_delta, output_delta

# def update_weights(X, hidden_layer_output, output_delta, hidden_layer_delta, W_h, W_o, learning_rate):
#     W_o += hidden_layer_output.T.dot(output_delta) * learning_rate
#     W_h += X.T.dot(hidden_layer_delta) * learning_rate

# def train_neural_network(X_train, y_train, epochs=10, hidden_size=25, learning_rate=0.01):
#     input_size = X_train.shape[1]
#     output_size = 1

#     W_h, W_o = initialize_parameters(input_size, hidden_size, output_size)

#     for epoch in range(epochs):
#         hidden_layer_output, output_layer_output = forward_propagation(X_train, W_h, W_o)

#         hidden_layer_delta, output_delta = backward_propagation(X_train, y_train.reshape(-1, 1), hidden_layer_output, output_layer_output, W_o)

#         update_weights(X_train, hidden_layer_output, output_delta, hidden_layer_delta, W_h, W_o, learning_rate)

#         loss = np.mean(np.square(y_train.reshape(-1, 1) - output_layer_output))
#         print(f"Epoch {epoch}, Loss: {loss}")

#     return W_h, W_o

# def predict(X, W_h, W_o):
#     _, output_layer_output = forward_propagation(X, W_h, W_o)
#     return (output_layer_output > 0.5).astype(int).flatten()

# # Train the neural network
# W_h, W_o = train_neural_network(X_train, y_train, epochs=10, hidden_size=int(2/3 * X_train.shape[1] + 2), learning_rate=0.01)

# # Test the neural network
# y_pred = predict(X_test, W_h, W_o)

# # Evaluate the accuracy
# accuracy = np.mean(y_pred == y_test)
# print(f"Accuracy: {accuracy}")

In [24]:
class Sigmoid:
    def forward(self, x):
        return 1 / (1 + np.exp(-x))

    def backward(self, x):
        return x * (1 - x)
    

class MeanSquaredError:
    def calculate(self, y_true, y_pred):
        return np.mean(np.square(y_true - y_pred))
    

class MLP:
    def __init__(self, input_size, hidden_size, output_size, activation, loss_function):
        np.random.seed(18)
        self.W_h = np.random.rand(input_size, hidden_size)
        self.W_o = np.random.rand(hidden_size, output_size)
        self.activation = activation
        self.loss_function = loss_function

    def forward_propagation(self, X):
        self.hidden_layer_input = np.dot(X, self.W_h)
        self.hidden_layer_output = self.activation.forward(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.W_o)
        self.output_layer_output = self.activation.forward(self.output_layer_input)

    def backward_propagation(self, X, y):
        output_error = y - self.output_layer_output
        output_delta = output_error * self.activation.backward(self.output_layer_output)

        hidden_layer_error = output_delta.dot(self.W_o.T)
        hidden_layer_delta = hidden_layer_error * self.activation.backward(self.hidden_layer_output)

        return hidden_layer_delta, output_delta

    def update_weights(self, X, hidden_layer_delta, output_delta, learning_rate):
        self.W_o += self.hidden_layer_output.T.dot(output_delta) * learning_rate
        self.W_h += X.T.dot(hidden_layer_delta) * learning_rate

    def predict(self, X):
        self.forward_propagation(X)
        return (self.output_layer_output > 0.5).astype(int).flatten()
    
class Trainer:
    def __init__(self, neural_network):
        self.neural_network = neural_network

    def train(self, X_train, y_train, epochs=10, learning_rate=0.01):
        for epoch in range(epochs):
            self.neural_network.forward_propagation(X_train)
            hidden_layer_delta, output_delta = self.neural_network.backward_propagation(X_train, y_train.reshape(-1, 1))
            self.neural_network.update_weights(X_train, hidden_layer_delta, output_delta, learning_rate)

            loss = self.neural_network.loss_function.calculate(y_train.reshape(-1, 1), self.neural_network.output_layer_output)
            print(f"Epoch {epoch}, Loss: {loss}")

    def evaluate_accuracy(self, X_test, y_test):
        y_pred = self.neural_network.predict(X_test)
        accuracy = np.mean(y_pred == y_test)
        print(f"Accuracy: {accuracy}")


activation = Sigmoid()
loss_function = MeanSquaredError()

input_size = X_train.shape[1]
hidden_size = int(2 / 3 * input_size + 2)
output_size = 1
neural_network = MLP(input_size, hidden_size, output_size, activation, loss_function)

trainer = Trainer(neural_network)

trainer.train(X_train, y_train, epochs=10, learning_rate=0.01)

trainer.evaluate_accuracy(X_test, y_test)


Epoch 0, Loss: 0.6173457853015846
Epoch 1, Loss: 0.6103499674528282
Epoch 2, Loss: 0.5986254428179401
Epoch 3, Loss: 0.5762474544319833
Epoch 4, Loss: 0.5249733138496462
Epoch 5, Loss: 0.3880353133682924
Epoch 6, Loss: 0.23014577469595807
Epoch 7, Loss: 0.2299299737175882
Epoch 8, Loss: 0.22991669138125662
Epoch 9, Loss: 0.2299112922679349
Accuracy: 0.5909090909090909
